<a href="https://colab.research.google.com/github/snowdenrise/Data_Analysis_Distribution_Chart/blob/main/Data_Analysis_Distribution_Chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Press Ctrl + F9 if you want to run it all at once*

#Run this first! 🙂

In [ ]:
#Refactoring coding - Jonatan Serna



#Importing Libraries
import warnings
import numpy as np
import matplotlib.pylab as plt
import scipy as sc
import pandas as pd
from scipy.stats import nbinom, poisson, geom, norm

from google.colab import files

#Fitting functions

def mean_confidence_interval(data, confidence=0.95):
      a = 1.0 * np.array(data)
      n = len(a)
      m, se = np.mean(a), sc.stats.sem(a)
      h = se * sc.stats.t.ppf((1 + confidence) / 2., n-1)
      return m, m-h, m+h

def fit_continuous(data,discrete=False):
  if(discrete):
    y,x = np.histogram(data,density=True)
    x = (x+np.roll(x,-1))[:-1]/2.0

    best_distributions=[]
    distributions = ['norm','t']

    try:
      with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        for distribution in distributions:
          dist = getattr(sc.stats,distribution)
          params = dist.fit(data)


          #Separating the parameters
          params_sep=params[:-2]
          loc=params[-2]
          scale=params[-1]

          #PDF and error

          pdf=dist.pdf(x,loc=loc,scale=scale, *params_sep)
          sse = np.sum(np.power(y-pdf,2.0))
          

          best_distributions.append((distribution,list(params),sse))
    except Exceptions:
      pass
    
    sorted_best_dist=sorted(best_distributions,key=lambda x:x[2])
    if(sorted_best_dist[0][2]<=0.2):
      data_stt = pd.Series(data)
      print("Dataset Information")
      print("Mean: ",data_stt.mean())
      print("Sd: ",np.sqrt(data_stt.var()))

      print("Fit Information")
      print("Best Fit: ",sorted_best_dist[0][0])
      print("Best Fit params: ",sorted_best_dist[0][1])
      print("Best Fit SSE: ",sorted_best_dist[0][2])
      interval = mean_confidence_interval(data)
      print("Mean Confidence Interval (95%)")
      print("[",interval[1],";", interval[2],"]")

      best_dist=getattr(sc.stats,sorted_best_dist[0][0])
      best_params=sorted_best_dist[0][1]

      # Separate best parameters
      best_arg = best_params[:-2]
      best_loc = best_params[-2]
      best_scale = best_params[-1]

      # Get sane start and end points of distribution
      best_start = best_dist.ppf(0.01, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.01, loc=best_loc, scale=best_scale)
      best_end = best_dist.ppf(0.99, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.99, loc=best_loc, scale=best_scale)

      # Build PDF and turn into pandas Series
      best_x = np.linspace(best_start, best_end, num=10000)
      best_y = best_dist.pdf(best_x,loc=best_loc, scale=best_scale, *best_arg)
      best_pdf = pd.Series(best_y, best_x)

      plt.figure(figsize=(8,8))
      plt.title('Best distribution " '+sorted_best_dist[0][0]+' "')
      plt.hist(data,density=True)
      best_pdf.plot(lw=2,label="Best Fit",legend=True)
      return True
    else:
      return False
  else:
    data_stt = pd.Series(data)
    print("Dataset Information")
    print("Mean: ",data_stt.mean())
    print("Sd: ",np.sqrt(data_stt.var()))

    y,x = np.histogram(data,density=True)
    x = (x+np.roll(x,-1))[:-1]/2.0

    best_distributions=[]
    distributions = ['norm','t','expon','gamma']

    try:
      with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        for distribution in distributions:
          dist = getattr(sc.stats,distribution)
          params = dist.fit(data)


          #Separating the parameters
          params_sep=params[:-2]
          loc=params[-2]
          scale=params[-1]

          #PDF and error

          pdf=dist.pdf(x,loc=loc,scale=scale, *params_sep)
          sse = np.sum(np.power(y-pdf,2.0))

          best_distributions.append((distribution,list(params),sse))
    except Exceptions:
      pass

    sorted_best_dist=sorted(best_distributions,key=lambda x:x[2])
    print("Fit Information")
    print("Best Fit: ",sorted_best_dist[0][0])
    print("Best Fit params: ",sorted_best_dist[0][1])
    print("Best Fit SSE: ",sorted_best_dist[0][2])

    best_dist=getattr(sc.stats,sorted_best_dist[0][0])
    best_params=sorted_best_dist[0][1]

    # Separate best parameters
    best_arg = best_params[:-2]
    best_loc = best_params[-2]
    best_scale = best_params[-1]

    # Get sane start and end points of distribution
    best_start = best_dist.ppf(0.01, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.01, loc=best_loc, scale=best_scale)
    best_end = best_dist.ppf(0.99, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.99, loc=best_loc, scale=best_scale)

    # Build PDF and turn into pandas Series
    best_x = np.linspace(best_start, best_end, num=10000)
    best_y = best_dist.pdf(best_x,loc=best_loc, scale=best_scale, *best_arg)
    best_pdf = pd.Series(best_y, best_x)

    plt.figure(figsize=(8,8))
    plt.title('Best distribution " '+sorted_best_dist[0][0]+' "')
    plt.hist(data,density=True)
    best_pdf.plot(lw=2,label="Best Fit",legend=True)
    interval = mean_confidence_interval(data)
    print("Mean Confidence Interval (95%)")
    print("[",interval[1],";", interval[2],"]")


def fit_discrete(data,ignore=False):
  
  if(ignore):
    data_try=False
  else:
    data_try=fit_continuous(data,discrete=True)

  if (data_try==False):
    n=len(data)
    data_fit=pd.Series(data)
    print("Dataset Information")

    mean = data_fit.mean()
    var = data_fit.var()

    print("Mean: ",mean)
    print("Sd: ",np.sqrt(var))

    likelihoods = {}
    params = {}
    try:
      with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        #Negative Binomial distribution

        p_nb = mean/var
        r_nb = p_nb*mean / (1-p_nb) 

        likelihoods['nbinomial'] = np.sum(np.log(np.array(data_fit.map(lambda val: nbinom.pmf(val, r_nb, p_nb)))))
        params["nbinomial"] = [r_nb,p_nb]

        #Poisson Distribution

        lambda_poisson = mean

        likelihoods['poisson'] = np.sum(np.log(np.array(data_fit.map(lambda val: poisson.pmf(val, lambda_poisson)))))
        params["poisson"] = lambda_poisson

        #Geometric Distribution

        p_geo=1/mean

        likelihoods['geometric'] = np.sum(np.log(np.array(data_fit.map(lambda val: geom.pmf(val, p_geo)))))
        params["geometric"] = p_geo
  
    except Exception:
      pass

    #Start printing the information
    
    likelihoods = {k: likelihoods[k] for k in likelihoods if not np.isnan(likelihoods[k])}

    best_fit = max(likelihoods, key=lambda x: likelihoods[x])
    print("Best Fit Information")
    print("Best fit:", best_fit)
    print("Log Likelihood:", likelihoods[best_fit])
    print("Parameters:", params[best_fit])

    interval = mean_confidence_interval(data)
    print("Mean Confidence Interval (95%)")
    print("[",interval[1],";", interval[2],"]")
    print("\n\n")

    plt.figure(figsize=(8,8))

    plt.hist(data,20,density=True)
    k = np.arange(data_fit.min(),data_fit.max()+1)
    plt.plot(k,poisson.pmf(k,lambda_poisson),'go', markersize=5,label="Poisson")
    plt.plot(k,nbinom.pmf(k,r_nb,p_nb),'go', markersize=5,color="red",label="Negative Binomial")
    plt.plot(k,geom.pmf(k,p_geo),'go', markersize=5,color="orange",label="Geometric")

    plt.legend()



def fit_distribution(data_name,column, continuous=False,ignore=False):
#Assigning uploaded CSV to the dataframe variable
  data_f=pd.read_csv(data_name)
#Changing Columns to Upper Case
  column = column.upper()
  data_f.columns = data_f.columns.str.upper()

  data=data_f[column]
  if(continuous):
    fit_continuous(data)
  else:
    fit_discrete(data,ignore)

def fit_distribution_test(data, continuous=False,ignore=False):
  if(continuous):
    fit_continuous(data)
  else:
    fit_discrete(data,ignore)

#Remove below # in the comments if you want to test the code
#print("Test Below ............ /n")
#data_1= [12,12,12,12,13,13,13,13,15,14,14,14,14,14,15,15,15,15,15,15,16,16,16,16,18,18,18,18,17,17,17,17,17,17,]
#fit_distribution_test(data_1)


#-----------------------------------------------------------------------------------------------------------------------
#Debes subir el dataset al mismo directorio, si estas desde Google Colab, Deberia ser en /content/
#debe tener las columnas, Date, Employee, y el dato totalizado por fecha, en este ejemplo, Bookings
#Guardar como CSV
#Se deben correr las primeras lineas para cargar los modulos y librerias

dataset = ""
columnAnalyze = ""
datasetOk = ""

def printingColumns():
  def extract_columns(df_index):
      # Convertir el objeto Index en una lista de Python
      column_list = df_index.tolist()

      # Procesar cada elemento de la lista para extraer el nombre de la columna
      column_names = []
      for col in column_list:
          col_name = col.replace('\nAvailable columns: ', '')
          column_names.append(col_name)

      return column_names
  # Crear el objeto Index
  index = pd.Index(['\nAvailable columns: Week', '\nAvailable columns: Year',
        '\nAvailable columns: Date', '\nAvailable columns: Bookings',
        '\nAvailable columns: Profit', '\nAvailable columns: Profit %'],
        dtype='object')

  # Llamar a la función extract_columns()
  column_names = extract_columns(index)

  # Imprimir los nombres de las columnas
  print("\nAvailable columns:\n")
  print(column_names)
  return

def runningTheBellCurve():
  datasetOk = input("\n\nHave you uploaded the dataset already?, YES or NO  🤔>>>>>>        \n")
  datasetOk = datasetOk.lower()
  if (datasetOk == "yes" or datasetOk == "y" or datasetOk == "si"):
    print("\n------------------------------------------------\nNice 👍, thanks, now:  \n")
    dataset = input("\n\nWhat is the full name (including the extension), of the dataset, make sure you save it in the same folder (possible '/content/' if you are using Google Colab) as a CSV File 📂  >>>>>>      \n") 
    printingColumns()
    columnAnalyze = input("\n\n What is the name of the column to analyze 📊 ?, please enter the SAME name, this is Case Sensitive 🔠🔡 >>>>>>      \n\n") 
    print("\n------------------------------------------------\n#Running 🚀\n------------------------------------------------\nDistribution Chart\n ⏬         ⏬         ⏬\n------------------------------------------------\n")
    fit_distribution(dataset,columnAnalyze)
  else:
    print("\n\n Please upload the dataset as a CSV File, \n\n Preferably, name it as data.csv, it would be easier for both of us 😊 \n\n")
    uploaded = files.upload()
    for fn in uploaded.keys():
      print("------------------------------------------------------------------------------")
      print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
      print("------------------------------------------------------------------------------")
      dataset = '{name}'.format(name=fn) #Assigning the name of the just uploaded document
    #Assigning (temporary) uploaded CSV to the dataframe variable
    data_f=pd.read_csv(dataset)
    #print("\nAvailable columns: "+data_f.columns) #Printing Columns
    printingColumns()
    columnAnalyze = input("\n\n What is the name of the column to analyze 📊 ?, please enter the SAME name, this is Case Sensitive 🔠🔡 >>>>>>      \n ") 
    print("\n------------------------------------------------\n#Running 🚀\n------------------------------------------------\nDistribution Chart\n ⏬         ⏬         ⏬\n------------------------------------------------\n")
    fit_distribution(dataset,columnAnalyze)
  return


#Running the Bell Curve NOW!!!

#runningTheBellCurve()

#Now run this 🚀

In [ ]:
runningTheBellCurve()

# Manual visual with Seaborn 🌊

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

%matplotlib inline

#sns.set(rc={'figure.figsize':(11.7,8.27)})

df_train = pd.read_csv("data_loadsmith.csv")

sns.distplot(df_train["Bookings"]) ##Distribution Plot

#sns.set(font_scale=1.25) 

#    *hm = sns.heatmap(cm, cbar=True, annot=True, fmt=".2f", annot_kws={"size":10}, yticklabels=cols.values, xticklabels=cols.values)
#   plt.show()**/

Refactoring

In [ ]:
# Refactoring coding - Jonatan Serna

import warnings
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
from google.colab import files

# Define functions
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def fit_continuous(data, discrete=False):
    # Get dataset statistics
    data_stt = pd.Series(data)
    print("Dataset Information")
    print("Mean: ", data_stt.mean())
    print("Sd: ", np.sqrt(data_stt.var()))

    # Create histogram
    y, x = np.histogram(data, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Fit distributions and find best one
    best_distributions = []
    distributions = ['norm', 't', 'expon', 'gamma'] if not discrete else ['norm', 't']

    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            for distribution in distributions:
                dist = getattr(stats, distribution)
                params = dist.fit(data)

                #Separating the parameters
                params_sep = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate PDF and error
                pdf = dist.pdf(x, loc=loc, scale=scale, *params_sep)
                sse = np.sum(np.power(y - pdf, 2.0))

                best_distributions.append((distribution, list(params), sse))
    except Exception as e:
        print(e)
        return False

    sorted_best_dist = sorted(best_distributions, key=lambda x: x[2])
    print("Fit Information")
    print("Best Fit: ", sorted_best_dist[0][0])
    print("Best Fit params: ", sorted_best_dist[0][1])
    print("Best Fit SSE: ", sorted_best_dist[0][2])

    # Plot histogram and best fit
    best_dist = getattr(stats, sorted_best_dist[0][0])
    best_params = sorted_best_dist[0][1]

    # Separate best parameters
    best_arg = best_params[:-2]
    best_loc = best_params[-2]
    best_scale = best_params[-1]

    # Get sane start and end points of distribution
    best_start = best_dist.ppf(0.01, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.01, loc=best_loc, scale=best_scale)
    best_end = best_dist.ppf(0.99, *best_arg, loc=best_loc, scale=best_scale) if best_arg else best_dist.ppf(0.99, loc=best_loc, scale=best_scale)

    # Build PDF and turn into pandas Series
    best_x = np.linspace(best_start, best_end, num=10000)
    best_y = best_dist.pdf(best_x, loc=best_loc, scale=best_scale, *best_arg)
    best_pdf = pd.Series(best_y, best_x)

    plt.figure(figsize=(8,8))
    plt.title('Best distribution "' + sorted_best_dist[0][0] + '"')
    plt
